In [15]:
import pandas as pd
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import os
import csv
from preprocessing import *
from scipy import stats as st

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.nn.functional as F
from torch.autograd import Variable
import math
from sklearn.manifold import TSNE

In [5]:
# The 19 activities are:
# sitting (0): index 0,
# standing (1),
# lying on back and on right side (2 and 3),
# ascending and descending stairs (4 and 5),
# standing in an elevator still (6)
# and moving around in an elevator (7),
# walking in a parking lot (8),
# walking on a treadmill with a speed of 4 km/h (in flat and 15 deg inclined positions) (9 and 10),
# running on a treadmill with a speed of 8 km/h (11),
# exercising on a stepper (12),
# exercising on a cross trainer (13),
# cycling on an exercise bike in horizontal and vertical positions (14 and 15),
# rowing (16),
# jumping (17),
# and playing basketball (18): index 18

activities = [0, 2, 3, 1, 8, 4, 14, 15, 14, 15, 1, 8, 11, 1, 12, 18, 5]
durations = [10, 5, 7, 3, 30, 8, 25, 25, 25, 20, 10, 30, 30, 4, 40, 80, 8]
person = 1

time_series, labels = get_daily_sports_timeseries('.\\Data\\Daily Sports Activities\\', person, activities, durations)

In [11]:
time_series[:,75:250].shape

(45, 175)

In [13]:
windowLength = 75
overlapPercent = 0.0
activity_names = ['sitting',
                 'standing',
                 'lying on back',
                 'lying on side',
                 'ascending stairs',
                 'descending stairs',
                 'standing in elevator',
                 'moving in elevator',
                 'walking in lot',
                 'walking on treadmill flat'
                 'walking on treadmill incline',
                 'running on treadmill',
                 'exercising on stepper',
                 'exercising on cross trainer',
                 'cycling on bike horizontal',
                 'cycling on bike vertical',
                 'rowing',
                 'jumping',
                 'basketball']


start = 0
subsequences = []
subsequence_labels = []

while start+windowLength < len(time_series[0]):
    subsequence_labels.append(st.mode(labels[start:start+windowLength]))    
    subsequence = time_series[:,start:start+windowLength]
    start = start+windowLength - int(overlapPercent*windowLength)
    subsequences.append(subsequence)

subsequences = np.asarray(subsequences)
subsequence_labels = np.asarray(subsequence_labels)
print(len(subsequences))
print(len(subsequence_labels))

10199
10199


In [16]:
def loss_fn(recon_z, x):
    BCE = F.mse_loss(recon_z, x)
    return BCE

def get_dataloader(data, batch_size, num_workers=0, data_transforms=None):
    if data_transforms is None:
        data_transforms = transforms.ToTensor()
        
    data_tensor = torch.from_numpy(data)
    
    data_loader = DataLoader(dataset=data_tensor, 
                             batch_size=batch_size, 
                             num_workers=num_workers)
    
    return data_loader

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

data_loader = get_dataloader(data=subsequences,
                          batch_size=50,
                          num_workers=2,
                          data_transforms=None)

cuda


In [17]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class UnFlatten(nn.Module):
    def forward(self, input, size1=32, size2=5):
        return input.view(input.size(0), size1, size2)
    
class Autoencoder(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.clustering_space=None
        
        self.encoder = nn.Sequential(
#             nn.Flatten(),
#             nn.Linear(75, 60),
#             nn.LeakyReLU(0.1),
#             nn.Linear(60, 45),
#             nn.LeakyReLU(0.1),
#             nn.Linear(45, 30),
#             nn.LeakyReLU(0.1),
#             nn.Linear(2000, 1500),
#             nn.LeakyReLU(0.1)
            nn.Conv1d(45, 45, 6, stride=4), # increase chan
            nn.LeakyReLU(0.1),
            nn.Conv1d(45, 45, 5, stride=3),
            nn.LeakyReLU(0.1),
            nn.Conv1d(45, 45, 4, stride=2),
            nn.LeakyReLU(0.1),
#             nn.Conv1d(16, 32, 3, stride=2),
#             nn.LeakyReLU(0.1),
            #nn.Flatten()
        )
        
        #self.fc1 = nn.Linear(2925, 2200)
        #self.fc2 = nn.Linear(2200, 1500)
        #self.fc3 = nn.Linear(1500, 2200)
        #self.fc4 = nn.Linear(2200, 2925)
        
        self.decoder = nn.Sequential(
#             nn.ConvTranspose1d(1, 1, 3, stride=1),
#             nn.LeakyReLU(0.1),
#             nn.ConvTranspose1d(1, 1, 3, stride=1),
#             nn.LeakyReLU(0.1),
#             nn.ConvTranspose1d(1, 1, 5, stride=2,output_padding=1),
#             nn.LeakyReLU(0.1),
#             nn.ConvTranspose1d(1, 1, 5, stride=2),
#             nn.LeakyReLU(0.1)
#             nn.Linear(30, 45),
#             nn.LeakyReLU(0.1),
#             nn.Linear(45, 60),
#             nn.LeakyReLU(0.1),
#             nn.Linear(60, 75),
#             nn.LeakyReLU(0.1),
#             nn.Linear(3000, 3375),
#             nn.LeakyReLU(0.1),
            #UnFlatten(),
            nn.ConvTranspose1d(45, 45, 4, stride=2),
            nn.LeakyReLU(0.1),
            nn.ConvTranspose1d(45, 45, 5, stride=3),
            nn.LeakyReLU(0.1),
            nn.ConvTranspose1d(45, 45, 6, stride=4),
            nn.LeakyReLU(0.1),
            nn.Upsample(size=75, mode='nearest')
#             nn.ConvTranspose1d(4, 1, 5, stride=2),
#             nn.LeakyReLU(0.1),
        )
        
        
    def encode(self, x):
        h = self.encoder(x)
        #h = self.fc1(h)
        return h
    
    def decode(self, z):
        #z = self.fc3(z)
        #z = self.fc4(z)
        z = self.decoder(z)
        return z
    
    def forward(self, x):
        z = self.encode(x)
        #z = self.fc2(z)
        z = self.decode(z)
        return z

In [18]:
model = Autoencoder()
model = model.double()

criterion = loss_fn
optimizer = torch.optim.Adam(model.parameters(), lr=1.5e-4)

In [19]:
dataiter = iter(data_loader)
data = dataiter.next()
features = data
print(features.shape)

# Point to training loop video
num_epochs = 300
outputs = []
for epoch in range(num_epochs):
    for ts in data_loader:
        # img = img.reshape(-1, 28*28) # -> use for Autoencoder_Linear
        recon = model(ts.double())
        loss = criterion(recon, ts)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')
    outputs.append((epoch, ts, recon))

torch.Size([50, 45, 75])
Epoch:1, Loss:6.1007
Epoch:2, Loss:4.8361


KeyboardInterrupt: 